# **Pre-processing images to be used in the YMCA**
**Jupyter notebook développé pour le pre-processing des images Sentinel-2 et Sentinel-1 pour les utiliser plus tard dans un modèle deep learning.** Ce notebook a été réalisé par Julien Govoorts dans le cardre d'un stage au sein de l'ANAGEO (IGEAT-ULB) pour le cours de STAG-F026 au sein du master en sciences et gestion de l'environnement à finalité sciences de l'environnement. Ce notebook utilise des données Sentinel-1 et 2 dans le but, dans un second temps, de cartographier (selon une probabilité) à partir d'un modèle deep learning les bidonvilles dans Nairobi (Kenya). 

Auteurs : Julien Govoorts\
Superviseur : Taïs Grippa

In [1]:
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, concatenate
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dot, Add

In [2]:
from tensorflow.keras.layers import BatchNormalization

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.optimizers import SGD, Adam

In [5]:
import tensorflow as tf

In [6]:
def import_hf5(data_path, data_name, get_name):
    hf = h5py.File(os.path.join(data_path, data_name), mode="r")
    data = hf.get(get_name)
    stack = np.array(data)
    hf.close()
    return stack

In [7]:
#code from https://github.com/tgrippa/Partimap/blob/cd72ecd2f3e32794eb3e0ee70399de1811b5399e/Notebooks/Data_preprocessing.ipynb
def normalise_01(image_data):
    stack = image_data
    image_data -= np.min(stack, axis=0)
    image_data /= (np.max(stack, axis=0) - np.min(stack, axis=0))
    return image_data

In [8]:
def standardise(image_data):
    image_data -= np.mean(image_data, axis=0, dtype=np.float32)
    image_data /= np.std(image_data, axis=0, dtype=np.float32)
    return image_data

In [9]:
def make_csv(predict, patch_id_raster, name):
    import csv

    predict = predict.reshape(1,predict.size).tolist()[0]

    header = ['prob_slum','id']
    cont = []
    cont.append(header)
    for x,y in zip(predict,patch_id_raster):
        current_row = [x,y[0]]
        cont.append(current_row)

    cont

    with open(str(name)+'.csv','w') as file :
        write = csv.writer(file)
        write.writerows(cont)

In [10]:
def reshape(list_input):
    L = np.array(list_input)
    L = L.reshape(L.size,-1)
    
    return L

In [11]:
import h5py
import os, sys

In [12]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

### Importing modules from SRC

In [13]:
# Add local module to the path
src = os.path.abspath('SRC/')
if src not in sys.path:
    sys.path.append(src)


### Importing modules from Modules

In [14]:
# Add local module to the path
src = os.path.abspath('Modules/')
if src not in sys.path:
    sys.path.append(src)


In [15]:
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that checks and creates folder
from mkdir import check_create_dir
# Import functions for plots
from plots import plot_loss, plot_pred_test, plot_pred_train, plot_loss_multirun
# Import functions for metrics to monitoring accuracy
from metrics import coeff_determination
# Import functions for display
from display import Norma_Xpercentile
# Import functions for exportation of results
from export_results import save_predictions, write_run_metrics_file
# Import LRFinder function
from LRFinder import LRFinder

In [16]:
from chunckage import chunckage

from MERGE import merge_RGBNIR

from PlotHistories import plotandcsv_history_kfold

from stackage import stackage

from CircleMaker import circle

from assimilation import assimilation

### Function to merge multiple single band raster images to one raster multiplebands 

In [9]:
merge_RGBNIR(["Images/big_raster/S2/B04_S2_2019Q1.tif","Images/big_raster/S2/B03_S2_2019Q1.tif","Images/big_raster/S2/B02_S2_2019Q1.tif",'Images/big_raster/S2/B08_S2_2019Q1.tif']
             ,"Images/big_raster/",'RGBNIR','.tif')

In [8]:
merge_RGBNIR(["Images/big_raster/S2/B04_S2_2019Q1.tif","Images/big_raster/S2/B03_S2_2019Q1.tif","Images/big_raster/S2/B02_S2_2019Q1.tif"]
             ,"Images/big_raster/",'RGB','.tif')

In [11]:
merge_RGBNIR(["Images/big_raster/S2/B04_S2_2019Q1.tif","Images/big_raster/S2/B03_S2_2019Q1.tif","Images/big_raster/S2/B02_S2_2019Q1.tif",'Images/big_raster/S2/B05_S2_2019Q1.tif','Images/big_raster/S2/B06_S2_2019Q1.tif',
             'Images/big_raster/S2/B07_S2_2019Q1.tif','Images/big_raster/S2/B08_S2_2019Q1.tif','Images/big_raster/S2/B08A_S2_2019Q1.tif','Images/big_raster/S2/B011_S2_2019Q1.tif','Images/big_raster/S2/B012_S2_2019Q1.tif',
             'Images/big_raster/S1/s1_vv.tif','Images/big_raster/S1/s1_vh.tif','Images/big_raster/S1/s1_coherence.tif']
             ,"Images/big_raster/",'ALLS2S1','.tif')

In [8]:
merge_RGBNIR(["Images/big_raster/S1/s1_coherence.tif","Images/big_raster/S1/s1_vv.tif","Images/big_raster/S1/s1_vh.tif"]
             ,"Images/big_raster/",'ALLS1','.tif')

In [10]:
merge_RGBNIR(["Images/big_raster/S2/B02_S2_2019Q1.tif","Images/big_raster/S2/B08_S2_2019Q1.tif","Images/big_raster/S2/B8A_S2_2019Q1.tif",'Images/big_raster/S2/B12_S2_2019Q1.tif']
             ,"Images/big_raster/",'BNIR','.tif')

In [9]:
merge_RGBNIR(["Images/big_raster/S1/s1_vv.tif","Images/big_raster/S1/s1_vh.tif"]
             ,"Images/big_raster/",'VVVHS1','.tif')

## Importing and pre-processing data

In [35]:
S1_patch_path, S1_patch_id, S1_patch_lulc = chunckage("Images/big_raster/","VVVHS1",".tif",'Images/big_sample/nairobi_samples_binary/','nairobi_samples_binary.gpkg','tr_VVVHS1','cat','label',1)
S2_patch_path, S2_patch_id, S2_patch_lulc = chunckage("Images/big_raster/","RGBNIR",".tif",'Images/big_sample/nairobi_samples_binary/','nairobi_samples_binary.gpkg','tr_VVVHRGBNIR','cat','label',1)

(238890.0, 288890.0, 9846380.0, 9871680.0)
(238890.0, 288890.0, 9846380.0, 9871680.0)


In [36]:
S1_grid_path, S1_grid_id = chunckage("Images/big_raster/","VVVHS1",".tif",'Images/big_grid/','grid.gpkg','g_VVVHS1','id','lulc',0)
S2_grid_path, S2_grid_id = chunckage("Images/big_raster/","RGBNIR",".tif",'Images/big_grid/','grid.gpkg','g_VVVHRGBNIR','id','lulc',0)


(238890.0, 289090.0, 9846380.0, 9871680.0)
(238890.0, 289090.0, 9846380.0, 9871680.0)


In [37]:
data_path = 'Images/big_raster/tr_VVVHRGBNIR_patch/'
with h5py.File(os.path.join(data_path,"tr_Ymodel_VVVHRGBNIR.hdf5"), mode="a") as f:
    f["id_sar"] = S1_patch_id               
    f["id_rgb"] = S2_patch_id                 
    f["lulc_sar"] = S1_patch_lulc          
    f["lulc_rgb"] = S2_patch_lulc       
    f["patch_sar_path"] = S1_patch_path  
    f["patch_rgb_path"] = S2_patch_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/tr_VVVHRGBNIR_patch/


In [38]:
data_path = 'Images/big_raster/g_VVVHRGBNIR_patch/'
with h5py.File(os.path.join(data_path,"g_Ymodel_VVVHRGBNIR.hdf5"), mode="a") as f:
    f["id_sar"] = S1_grid_id               
    f["id_rgb"] = S2_grid_id                       
    f["patch_sar_path"] = S1_grid_path  
    f["patch_rgb_path"] = S2_grid_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/g_VVVHRGBNIR_patch/


In [39]:
S1_patch_path, S1_patch_id, S1_patch_lulc = chunckage("Images/big_raster/","VVVHS1",".tif",'Images/big_sample/nairobi_samples_binary/','nairobi_samples_binary.gpkg','tr_VVVHS1','cat','label',1)
S2_patch_path, S2_patch_id, S2_patch_lulc = chunckage("Images/big_raster/","BNIR",".tif",'Images/big_sample/nairobi_samples_binary/','nairobi_samples_binary.gpkg','tr_VVVHBNIR','cat','label',1)

(238890.0, 288890.0, 9846380.0, 9871680.0)
(238890.0, 288890.0, 9846380.0, 9871680.0)


In [40]:
S1_grid_path, S1_grid_id = chunckage("Images/big_raster/","VVVHS1",".tif",'Images/big_grid/','grid.gpkg','g_VVVHS1','id','lulc',0)
S2_grid_path, S2_grid_id = chunckage("Images/big_raster/","BNIR",".tif",'Images/big_grid/','grid.gpkg','g_VVVHBNIR','id','lulc',0)


(238890.0, 289090.0, 9846380.0, 9871680.0)
(238890.0, 289090.0, 9846380.0, 9871680.0)


In [41]:
data_path = 'Images/big_raster/tr_VVVHBNIR_patch/'
with h5py.File(os.path.join(data_path,"tr_Ymodel_VVVHBNIR.hdf5"), mode="a") as f:
    f["id_sar"] = S1_patch_id               
    f["id_rgb"] = S2_patch_id                 
    f["lulc_sar"] = S1_patch_lulc          
    f["lulc_rgb"] = S2_patch_lulc       
    f["patch_sar_path"] = S1_patch_path  
    f["patch_rgb_path"] = S2_patch_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/tr_VVVHBNIR_patch/


In [42]:
data_path = 'Images/big_raster/g_VVVHBNIR_patch/'
with h5py.File(os.path.join(data_path,"g_Ymodel_VVVHBNIR.hdf5"), mode="a") as f:
    f["id_sar"] = S1_grid_id               
    f["id_rgb"] = S2_grid_id                       
    f["patch_sar_path"] = S1_grid_path  
    f["patch_rgb_path"] = S2_grid_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/g_VVVHBNIR_patch/


In [43]:
S1_patch_path, S1_patch_id, S1_patch_lulc = chunckage("Images/big_raster/","VVVHS1",".tif",'Images/big_sample/nairobi_samples_binary/','nairobi_samples_binary.gpkg','tr_VVVHS1','cat','label',1)
S2_patch_path, S2_patch_id, S2_patch_lulc = chunckage("Images/big_raster/","RNIR",".tif",'Images/big_sample/nairobi_samples_binary/','nairobi_samples_binary.gpkg','tr_VVVHRNIR','cat','label',1)

(238890.0, 288890.0, 9846380.0, 9871680.0)
(238890.0, 288890.0, 9846380.0, 9871680.0)


In [44]:
S1_grid_path, S1_grid_id = chunckage("Images/big_raster/","VVVHS1",".tif",'Images/big_grid/','grid.gpkg','g_VVVHS1','id','lulc',0)
S2_grid_path, S2_grid_id = chunckage("Images/big_raster/","RNIR",".tif",'Images/big_grid/','grid.gpkg','g_VVVHRNIR','id','lulc',0)


(238890.0, 289090.0, 9846380.0, 9871680.0)
(238890.0, 289090.0, 9846380.0, 9871680.0)


In [45]:
data_path = 'Images/big_raster/tr_VVVHRNIR_patch/'
with h5py.File(os.path.join(data_path,"tr_Ymodel_VVVHRNIR.hdf5"), mode="a") as f:
    f["id_sar"] = S1_patch_id               
    f["id_rgb"] = S2_patch_id                 
    f["lulc_sar"] = S1_patch_lulc          
    f["lulc_rgb"] = S2_patch_lulc       
    f["patch_sar_path"] = S1_patch_path  
    f["patch_rgb_path"] = S2_patch_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/tr_VVVHRNIR_patch/


In [46]:
data_path = 'Images/big_raster/g_VVVHRNIR_patch/'
with h5py.File(os.path.join(data_path,"g_Ymodel_VVVHRNIR.hdf5"), mode="a") as f:
    f["id_sar"] = S1_grid_id               
    f["id_rgb"] = S2_grid_id                       
    f["patch_sar_path"] = S1_grid_path  
    f["patch_rgb_path"] = S2_grid_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/g_VVVHRNIR_patch/
